### Week 2 - Adding Sales Report - Exercise 1 Task 1

In [ ]:
USE `LittleLemonDB`;
CREATE  OR REPLACE VIEW `OrdersView` AS SELECT OrderID, Quantity, TotalCost FROM Orders WHERE Quantity > 2;

SELECT * FROM OrdersView;

### Week 2 - Adding Sales Report - Exercise 1 Task 2

In [ ]:
SELECT Customers.CustomerID, Customers.CustomerName, Orders.OrderID, Orders.TotalCost, Menus.MenuName, MenuItems.CourseName, MenuItems.StarterName FROM Customers
INNER JOIN Orders ON Customers.CustomerID = Orders.CustomerID
INNER JOIN Menus ON Orders.MenuID = Menus.MenuID
INNER JOIN MenuItems ON Menus.MenuItemID = MenuItems.MenuItemID
WHERE TotalCost > 150;

### Week 2 - Adding Sales Report - Exercise 1 Task 3

In [ ]:
Select MenuName FROM Menus WHERE MenuID IN (SELECT MenuID FROM Orders WHERE Quantity > 2);

### Week 2 - Adding Sales Report - Exercise 2 Task 1

In [ ]:
CREATE PROCEDURE `GetMaxQuantity`() SELECT MAX(Quantity) AS "Max Quantity in Order" FROM Orders;

CALL GetMaxQuantity();

### Week 2 - Adding Sales Report - Exercise 2 Task 2

In [ ]:
PREPARE GetOrderDetail FROM "SELECT OrderID, Quantity, TotalCost FROM Orders WHERE CustomerID = ?";

SET @id = 1;
EXECUTE GetOrderDetail USING @id;

### Week 2 - Adding Sales Report - Exercise 2 Task 3

In [ ]:
DROP PROCEDURE IF EXISTS CancelOrder;
DELIMITER //
CREATE PROCEDURE CancelOrder(IN order_id INT)
BEGIN
DELETE FROM Orders WHERE OrderId = order_id;
SELECT CONCAT("Order ", order_id, " is cancelled") AS Confirmation;
END //
DELIMITER ;

CALL CancelOrder(6);

### Week 2 - Table Booking System - Exercise 1 Task 1

In [ ]:
INSERT INTO Bookings (BookingDate, BookingTime, TableNumber, NumberOfGuests, CustomerID) 
VALUES 
("2022-10-10", "18:00:00", 5, 1, 1), 
("2022-11-12", "18:00:00", 3, 1, 3), 
("2022-10-11", "18:00:00", 2, 1, 2), 
("2022-10-13", "18:00:00", 2, 1, 1);

SELECT * FROM Bookings;

### Week 2 - Table Booking System - Exercise 1 Task 2

In [ ]:
DROP PROCEDURE IF EXISTS CheckBooking;

DELIMITER //
CREATE PROCEDURE CheckBooking(booking_date DATE, booking_time TIME, table_number INT)
BEGIN
DECLARE status_of_table VARCHAR(255) DEFAULT "is available";
IF table_number IN (SELECT TableNumber FROM Bookings WHERE BookingDate = booking_date AND BookingTime = booking_time) THEN SET status_of_table = "is already booked";
END IF;
SELECT CONCAT("Table ", table_number, " ", status_of_table, " for ", booking_time, " on ", booking_date) AS "Booking Status";
END //
DELIMITER ;

call CheckBooking("2022-11-12", "18:00:00", 3);

### Week 2 - Table Booking System - Exercise 1 Task 3

In [ ]:
DROP PROCEDURE IF EXISTS AddValidBooking;

DELIMITER //
CREATE PROCEDURE AddValidBooking(booking_date DATE, booking_time TIME, table_number INT, number_of_guests INT, customer_id INT)
BEGIN
DECLARE status_of_table VARCHAR(255) DEFAULT "is available - booking confirmed";
START TRANSACTION;
INSERT INTO Bookings (BookingDate, BookingTime, TableNumber, NumberOfGuests, CustomerID) 
VALUES (booking_date, booking_time, table_number, number_of_guests, customer_id);
IF (SELECT COUNT(customerID) FROM Bookings WHERE BookingDate = booking_date AND BookingTime = booking_time AND TableNumber = table_number) > 1 THEN
SET status_of_table = "is already booked - booking cancelled";
END IF;
IF status_of_table = "is already booked - booking cancelled" THEN ROLLBACK;
ELSE COMMIT;
END IF;
SELECT CONCAT("Table ", table_number, " ", status_of_table) AS "Booking Status";
END //
DELIMITER ;

CALL AddValidBooking("2022-10-10", "18:00:00", 5, 2, 2);

### Week 2 - Table Booking System - Exercise 2 Task 1

In [ ]:
DROP PROCEDURE IF EXISTS AddBooking;

DELIMITER //
CREATE PROCEDURE AddBooking(booking_date DATE, booking_time TIME, table_number INT, number_of_guests INT, customer_id INT)
BEGIN
DECLARE booking_id INT;
INSERT INTO Bookings (BookingDate, BookingTime, TableNumber, NumberOfGuests, CustomerID) 
VALUES (booking_date, booking_time, table_number, number_of_guests, customer_id);
SET booking_id = (SELECT BookingID FROM Bookings WHERE BookingDate = booking_date AND BookingTime = booking_time AND TableNumber = table_number AND NumberOfGuests = number_of_guests AND CustomerID = customer_id);
SELECT CONCAT("New booking ", booking_id, " confirmed for table ", table_number, " for ", number_of_guests, " guests at ", booking_time, " on ", booking_date) AS Confirmation;
END //
DELIMITER ;

CALL AddBooking("2022-12-30", "18:00:00", 4, 2, 3);

### Week 2 - Table Booking System - Exercise 2 Task 2

In [ ]:
DROP PROCEDURE IF EXISTS UpdateBooking;

DELIMITER //
CREATE PROCEDURE UpdateBooking(booking_id INT, booking_date DATE, booking_time TIME, table_number INT, number_of_guests INT)
BEGIN
UPDATE Bookings SET BookingDate = booking_date, BookingTime = booking_time, TableNumber = table_number, NumberOfGuests = number_of_guests WHERE BookingID = booking_id;
SELECT CONCAT("Booking ", booking_id, " updated") AS Confirmation;
END //
DELIMITER ;

CALL UpdateBooking(10, "2022-12-31", "18:00:00", 4, 2);

### Week 2 - Table Booking System - Exercise 2 Task 3

In [ ]:
DROP PROCEDURE IF EXISTS CancelBooking;

DELIMITER //
CREATE PROCEDURE CancelBooking(booking_id INT)
BEGIN
DELETE FROM Bookings WHERE BookingID = booking_id;
SELECT CONCAT("Booking ", booking_id, " cancelled") AS Confirmation;
END //
DELIMITER ;

CALL CancelBooking(10);